# Lab 4. Financial Advisory Hub

## Introduction

In this notebook, we'll create a Financial Advisory Hub using Amazon Bedrock's multi-agent collaboration feature. This hub will integrate the capabilities of our previously created agents:

1. **Data Analytics Agent (Lab 1)**: Provides financial data analysis, transaction history, and financial projections
2. **Customer Insights Agent (Lab 2)**: Explains financial visualizations and recommends financial products

[Multi-agent Collaboration](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-multi-agents-collaboration.html) is an Amazon Bedrock Agents native capability that enables hierarchical collaboration between agents. You can enable agent collaboration and associate secondary agents to a supervisor one. These secondary agents can be any existing agent within the same account. This composable pattern allows you to build a chain of agents, as shown in the figure below.

![collaboration_hierarchy](img/multi-agent_flow.png)

In this lab, we will create a supervisor agent that integrates the capabilities of our Data Analytics and Customer Insights agents to provide comprehensive financial advisory services.

The following represents the architecture of our Financial Advisory Hub:

![Architecture](img/financial_advisory_hub.png)

This Lab is divided into two notebooks:

- [4.1_financial_advisory_hub.ipynb](4.1_financial_advisory_hub.ipynb): Contains the Agent Setup
- [4.2_financial_advisory_invocation.ipynb](4.2_financial_advisory_invocation.ipynb): Contains the Agent Invocation examples

## Setup

Make sure that your boto3 version is the latest one.

If not, return to [notebook 1](../1-data-analytics/1_data_analytics_agent.ipynb) and run the Setup block again.

In [ ]:
!pip freeze | grep boto3

Get your workshop ID / resource suffix. If not found, return to [notebook 1](../1-data-analytics/1_data_analytics_agent.ipynb) and run the Setup block again.

In [ ]:
import os

def get_workshop_id():
    workshop_id_file = '../.workshop_id'
    if os.path.exists(workshop_id_file):
        with open(workshop_id_file, 'r') as f:
            return f.read().strip()
    else:
        return None
    
workshop_id = get_workshop_id()
resource_suffix = f"{workshop_id}"

if workshop_id is None:
    print("No workshop ID found. Please run the Setup script in notebook 1.")
else:
    print("Your resource suffix is", resource_suffix)

### Creating the Financial Advisory Hub Agent

In this section, we'll declare global variables that will act as helpers throughout the notebook and create our supervisor agent.

In [ ]:
import boto3
import os
import json
import time
import sys
from datetime import datetime

# Get AWS account ID
sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity()["Account"]

# Define foundation model
agent_foundation_model = [
    'anthropic.claude-3-5-sonnet-20240620-v1:0',
    'anthropic.claude-3-sonnet-20240229-v1:0',
    'anthropic.claude-3-haiku-20240307-v1:0'
]

In [ ]:
financial_hub_agent_name = f"financial-hub-agent-{resource_suffix}"

financial_hub_agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{financial_hub_agent_name}'

### Importing helper functions

Now, let's import the helper functions from the utils directory.

In [ ]:
import sys

sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
agents = AgentsForAmazonBedrock()

### Create the Financial Advisory Hub agent

Now, let's create our supervisor agent that will coordinate between the Data Analytics and Customer Insights agents.

In [ ]:
financial_hub_agent = agents.create_agent(
    financial_hub_agent_name,
    "Financial Advisory Hub that integrates data analytics and customer insights capabilities",
    """
    You are a Financial Advisory Hub, a sophisticated AI assistant designed to provide comprehensive financial guidance by integrating data analytics and customer insights capabilities.
    
    Your primary responsibilities include:
    
    1. Financial Data Analysis:
       - Analyze customer transaction history and financial data
       - Generate financial projections and forecasts
       - Identify spending patterns and trends
    
    2. Visualization Explanation:
       - Interpret and explain financial visualizations
       - Provide insights on spending trends, investment allocations, cash flow, and budget performance
       - Translate complex financial data into clear, actionable insights
    
    3. Financial Product Recommendations:
       - Suggest appropriate financial products based on customer profiles and needs
       - Provide personalized recommendations for investment strategies, savings plans, and financial tools
    
    4. Integrated Advisory Services:
       - Combine data analysis with visualization insights to provide holistic financial advice
       - Help customers understand their financial situation and make informed decisions
       - Offer strategic guidance for financial planning and goal achievement
    
    When handling user requests:
    - For data analysis and transaction-related queries, delegate to the Data Analytics Agent
    - For visualization explanation and financial product recommendations, delegate to the Customer Insights Agent
    - For complex queries requiring both capabilities, coordinate between both agents and synthesize their responses
    - Always maintain a professional, helpful tone while making financial information accessible
    
    Remember that you are assisting bank operators who are helping customers, so frame your responses accordingly.
    """,
    agent_foundation_model,
    agent_collaboration='SUPERVISOR_ROUTER'
)

financial_hub_agent

### Retrieve Sub-Agent Information

Let's retrieve the information about our sub-agents from previous labs.

In [ ]:
%store -r
analytics_agent_alias_arn, insights_agent_alias_arn

In [ ]:
# If store fails, add Alias ARNs here
#analytics_agent_alias_arn = ''
#insights_agent_alias_arn = ''

In [ ]:
sub_agents_list = [
    {
        'sub_agent_alias_arn': analytics_agent_alias_arn,
        'sub_agent_instruction': """Delegate financial data analysis tasks to the Data Analytics Agent. This agent can analyze transaction history, generate financial projections, and identify spending patterns. Use this agent for queries related to transaction data, financial statistics, and numerical analysis.""",
        'sub_agent_association_name': 'DataAnalyticsAgent',
        'relay_conversation_history': 'TO_COLLABORATOR'
    },
    {
        'sub_agent_alias_arn': insights_agent_alias_arn,
        'sub_agent_instruction': """Assign visualization explanation and financial product recommendation tasks to the Customer Insights Agent. This agent can interpret spending trends, investment allocations, cash flow, and budget performance visualizations. It can also recommend financial products based on customer profiles. Use this agent for queries related to explaining financial visualizations and providing product recommendations.""",
        'sub_agent_association_name': 'CustomerInsightsAgent',
        'relay_conversation_history': 'TO_COLLABORATOR'
    }
]

Check if parameters are correctly formatted:

In [ ]:
sub_agents_list

Associate all sub-agents with our Financial Advisory Hub agent:

In [ ]:
financial_hub_agent_alias_id, financial_hub_agent_alias_arn = agents.associate_sub_agents(
    financial_hub_agent[0], sub_agents_list
)

Check the response return to see all returned parameters:

In [ ]:
financial_hub_agent_alias_id, financial_hub_agent_alias_arn

In [ ]:
%store financial_hub_agent_alias_id
%store financial_hub_agent_alias_arn

## Next Steps
Congratulations! We've now created a Financial Advisory Hub agent that integrates the capabilities of our Data Analytics and Customer Insights agents. Next, let's invoke our agent to see it in action.